# __Homework 4:__ Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

---
The __actual sequences__ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
---


__The end of NA is__ `...CACGATAGATAAATAATAGTGCACCAT`
    
__The end of HA is__ `...CCGGATTTGCATATAATGATGCACCAT`

---    

    
The __sequencing reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA]-3'
or

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of NA]-3'

---   
    
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [41]:
# import needed modules
import re
import Bio.SeqIO
import Bio.Seq

# define a function to identify and return barcode sequence for a specified gene if matched or invalid
def read_barcode(seqread, bclen, gene_end, upstream='AGGCGGCCGC'):
    """Identify valid or invalid barcode with specified gene and known upstream sequence.
    
    Parameters
    ----------
    seqread : Seq object
        Nucleotide sequence read matching UPSTREAM-BARCODE in reverse orientation.
    bclen : int
        Length of barcode.
    gene_end : str
        Sequence corresponding to end of gene of interest.
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or `'invalid'` or None
        Sequence of the barcode in the forward orientation, 'invalid' if invalid barcode, or `None` if no match to expected barcoded sequence.

        
    """
    seq_upper_rev = str(seqread.upper().reverse_complement()) #convert Seq to str

    gene_search = re.compile(gene_end) #define search str for gene
    gene_match = gene_search.search(seq_upper_rev) #search Seq str for specified gene

    if gene_match:
        barcode_search = re.compile(gene_end + upstream + '(?P<barcode>[ATCG]{' +str(bclen)+ '})') #define search str for full sequencing read
        barcode_match = barcode_search.search(seq_upper_rev) #search Seq str for full read
        if barcode_match:
            barcode_seq = barcode_match.group('barcode') #pull barcode sequence as a str from Seq str
            return barcode_seq #return barcode sequence
        else:
            return "invalid" #return message if gene matches but no matching barcode
    else:
        return None #return `None` if no match to gene


# read sequencing data in the fastq file and convert into a list
reads = Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq')
seqreads = list(reads)

# create and populate list with sequences isolated from sequencing data
flu_seqs = []
for seqrecord in seqreads:
    flu_seqs.append(seqrecord.seq) #get Seq object from seqrecord and add to flu_seqs list

# assign variables to store ends of the HA and NA gene sequences in forward orientation
HA_end = 'CCGGATTTGCATATAATGATGCACCAT'
NA_end = 'CACGATAGATAAATAATAGTGCACCAT'

# CREATE DICT TO TRACK VALID AND INVALID BARCODES AND COUNTS FOR HA AND NA GENES
HA_barcode_counts = {} #create empty dict to track valid HA barcodes and counts
HA_num_reads = 0 #use to count total number of valid reads for HA gene
HA_invalid = 0 #use to track number of invalid reads for HA gene

NA_barcode_counts = {} #create empty dict to track valid NA barcodes and counts
NA_num_reads = 0 #use to count total number of valid reads for NA gene
NA_invalid = 0 #use to track number of invalid reads for NA gene

## track valid barcodes and count invalid reads for HA and NA genes
for seq in flu_seqs: #iterate along Seq sequences in flu_seqs list
    HA_barcode = read_barcode(seq, bclen=16, gene_end=HA_end) #identify barcode associated with HA sequence
    if HA_barcode: #if valid HA barcode
        if HA_barcode == "invalid":
            HA_invalid += 1 #add count to track total number of invalid reads mapped to HA
        elif HA_barcode in HA_barcode_counts:
            HA_barcode_counts[HA_barcode] += 1 #add count to existing dict value
            HA_num_reads += 1 #add count to track total number of reads mapped to HA
        else:
            HA_barcode_counts[HA_barcode] = 1 #create new dict key
            HA_num_reads += 1
    
    NA_barcode = read_barcode(seq, bclen=16, gene_end=NA_end) #identify barcode associated with NA sequence
    if NA_barcode:
        if NA_barcode == "invalid":
            NA_invalid += 1 #add count to track total number of invalid reads mapped to NA
        elif NA_barcode in NA_barcode_counts:
            NA_barcode_counts[NA_barcode] += 1 #add count to existing dict value
            NA_num_reads += 1 #add count to track total number of reads mapped to NA
        else:
            NA_barcode_counts[NA_barcode] = 1 #create new dict key
            NA_num_reads += 1

# print final valid read counts for HA and NA genes
print("Number of reads mapping to HA: " + str(HA_num_reads))
print("Number of reads mapping to NA: " + str(NA_num_reads))


Number of reads mapping to HA: 5249
Number of reads mapping to NA: 3909


2. How many HA sequences did not have a valid barcode? Also anwer the same question for NA.

In [44]:
# print counts of invalid barcodes as tracked from previous code cell
print('Number of HA sequences with invalid barcodes: ' + str(HA_invalid))
print('Number of NA sequences with invalid barcodes: ' + str(NA_invalid))

Number of HA sequences with invalid barcodes: 160
Number of NA sequences with invalid barcodes: 213


3. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [46]:
HA_max_counts = 0 #use to track maximum counts from HA_barcode_counts dict
HA_max_barcode = '' #use to track barcode with maximum counts from HA_barcode_counts dict

for HA_bc in HA_barcode_counts:
    if HA_barcode_counts[HA_bc] > HA_max_counts: #compare HA_bc count to existing maximum value
        HA_max_barcode = HA_bc #update barcode associated with max value
        HA_max_counts = HA_barcode_counts[HA_max_barcode] #update max value

# repeat above process to find NA barcode with most counts
NA_max_counts = 0 #use to track maximum counts from NA_barcode_counts dict
NA_max_barcode = '' #use to track barcode with maximum counts from NA_barcode_counts dict

for NA_bc in NA_barcode_counts:
    if NA_barcode_counts[NA_bc] > NA_max_counts: #compare NA_bc count to existing maximum value
        NA_max_barcode = NA_bc #update barcode associated with max value
        NA_max_counts = NA_barcode_counts[NA_max_barcode] #update max value

# print final 
print("The HA barcode '" + HA_max_barcode + "' has the most counts at " + str(HA_max_counts) + ".")
print("The NA barcode '" + NA_max_barcode + "' has the most counts at " + str(NA_max_counts) + ".")

The HA barcode 'CCCGACCCGACATTAA' has the most counts at 155.
The NA barcode 'ACCAGTTCTCCCCGGG' has the most counts at 152.
